##LSTMs for Human Activity Recognition Time Series Classification

Paper:  https://upcommons.upc.edu/bitstream/handle/2117/101769/IWAAL2012.pdf?sequence=1


In [ ]:
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
import tensorflow as tf  
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM
from keras.utils import to_categorical
from matplotlib import pyplot 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

# path of Drive folder
data_dir = '/gdrive/My Drive/Informazione_Multimediale/Colab_Notebooks/video/datasets/UCI_HAR/'

# Load data

There are 3 main signal types in the raw data: 
1. total acceleration
2. body acceleration
3. body gyroscope

Each has **3 axes** of data. This means that there are a total of **9 variables** for each time step.

Further, each series of data has been partitioned into **overlapping windows of 2.56 seconds** of data, or **128 time steps**. These windows of data correspond to the windows of engineered features (rows). This means that one row of data has  **128 * 9 = 1152** elements.

The signals are stored in the */Inertial Signals/* directory under the train and test subdirectories. Each axis of each signal is stored in a separate file, meaning that each of the train and test datasets have nine input files to load and one output file to load. We can batch the loading of these files into groups given the consistent directory structures and file naming conventions.

The input data is in CSV format where columns are separated by whitespace. Each of these files can be loaded as a NumPy array. The *load_file()* function below loads a dataset given the fill path to the file and returns the loaded data as a NumPy array.


In [ ]:
# load a single file as a numpy array
def load_file(filepath):
	dataframe = read_csv(filepath, header=None, delim_whitespace=True)
	return dataframe.values

We can then load all data for a given group (train or test) into a single three-dimensional NumPy array, where the dimensions of the array are **[samples, time steps, features]**.

To make this clearer, there are 128 time steps and nine features, where the number of samples is the number of rows in any given raw signal data file.

The *load_group()* function below implements this behavior. The *dstack()* NumPy function allows us to stack each of the loaded 3D arrays into a single 3D array where the variables are separated on the third dimension (features).



In [ ]:
# load a list of files and return as a 3d numpy array
def load_group(filenames, prefix=''):
	loaded = list()
	for name in filenames:
		data = load_file(prefix + name)
		loaded.append(data)
	# stack group so that features are the 3rd dimension
	loaded = dstack(loaded)
	return loaded

##Load all input signal data for a given group, such as train or test.

The load_dataset_group() function below loads all input signal data and the output data for a single group using the consistent naming conventions between the directories.

In [ ]:
# load a dataset train or test group
def load_dataset_group(path, group):
	# load all 9 files as a single array
  filenames = list()
  filepath = path + group + '/Inertial Signals/'

  # total acceleration
  filenames += ['total_acc_x_'+group+'.txt', 'total_acc_y_'+group+'.txt', 'total_acc_z_'+group+'.txt']
  # body acceleration
  filenames += ['body_acc_x_'+group+'.txt', 'body_acc_y_'+group+'.txt', 'body_acc_z_'+group+'.txt']
  # body gyroscope
  filenames += ['body_gyro_x_'+group+'.txt', 'body_gyro_y_'+group+'.txt', 'body_gyro_z_'+group+'.txt']

  # load input data
  X = load_group(filenames, filepath)
  # load class output
  y = load_file(path + group + '/y_'+group+'.txt')
  
  return X, y

The output data is defined as an integer for the class number. We must one hot encode these class integers so that the data is suitable for fitting a neural network multi-class classification model. We can do this by calling the to_categorical() Keras function.

The load_dataset() function below implements this behavior and returns the train and test X and y elements ready for fitting and evaluating the defined models.

In [ ]:
# load the dataset, returns train and test X and y elements
def load_dataset(path):
  # load all train

  trainX, trainy = load_dataset_group(path, 'train')
  print('Train X dim : ', trainX.shape)
  print('Train y dim : ', trainy.shape)

  # load all test
  testX, testy = load_dataset_group(path, 'test')
  print('Test X dim  : ', testX.shape)
  print('Test y dim  : ', testy.shape)

  # zero-offset class values
  trainy = trainy - 1
  testy = testy - 1

  # one hot encode y
  trainy = to_categorical(trainy)
  testy = to_categorical(testy)
  #print(trainX.shape, trainy.shape, testX.shape, testy.shape)
  return trainX, trainy, testX, testy

# Fit and Evaluate Model

First, we must define the LSTM model using the Keras deep learning library. The model requires a three-dimensional input with **[samples, time steps, features]**.

This is exactly how we have loaded the data, where one sample is one window of the time series data, each window has 128 time steps, and a time step has nine variables or features.

The output for the model will be a six-element vector containing the probability of a given window belonging to each of the six activity types.

Thees input and output dimensions are required when fitting the model, and we can extract them from the provided training dataset.
```
n_timesteps = trainX.shape[1]
n_features = trainX.shape[2]
n_outputs = trainy.shape[1]
```
The model is defined as a **Sequential Keras model**:

* the model has a **single** LSTM **hidden layer**
* a **dropout layer** (to reduce overfitting)
* a **dense fully connected hidden layer** (features extracted by the LSTM)
* final output layer is used to make predictions
```
model = Sequential()
model.add(LSTM(100, input_shape=(n_timesteps,n_features)))
model.add(Dropout(0.5))
model.add(Dense(100, activation='relu'))
model.add(Dense(n_outputs, activation='softmax'))
```

The model is fit for a **fixed number of epochs**, in **this case 15**, and a **batch size of 64 samples** will be used, where 64 windows of data will be exposed to the model before the weights of the model are updated.

Once the model is fit, it is evaluated on the test dataset and the accuracy of the fit model on the test dataset is returned.

Note, it is common to not shuffle sequence data when fitting an LSTM. Here we do shuffle the windows of input data during training (the default). In this problem, we are interested in harnessing the LSTMs ability to learn and extract features across the time steps in a window, not across windows.

The complete *evaluate_model()* function is listed below.

In [ ]:
# lstm model: fit and evaluate a model
def evaluate_model(trainX, trainy, testX, testy):
	verbose, epochs, batch_size = 0, 15, 64
	n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]
	model = Sequential()
	model.add(LSTM(100, input_shape=(n_timesteps,n_features)))
	model.add(Dropout(0.5))
	model.add(Dense(100, activation='relu'))
	model.add(Dense(n_outputs, activation='softmax'))
 
	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
	# fit network
	model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
	# evaluate model
	_, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
	return accuracy, model

# summarize scores
def summarize_results(scores):
	print(scores)
	m, s = mean(scores), std(scores)
	print('Accuracy: %.3f%% (+/-%.3f)' % (m, s))




The efficient Adam version of stochastic gradient descent may be used to optimize the network, and the categorical cross entropy loss function may be used given that we are learning a multi-class classification problem. Alternatively, it is possible to choose among the following

In [ ]:
# Optimizers https://www.tensorflow.org/api_docs/python/tf/keras/optimizers
adam = tf.keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
sgd = tf.keras.optimizers.SGD(lr=0.001, momentum=0.0, decay=0.0, nesterov=False)
adad = tf.keras.optimizers.Adadelta(lr=1.0,rho=0.95,epsilon=None,decay=0.0)
adag = tf.keras.optimizers.Adagrad(lr=0.01,epsilon=None,decay=0.0)
adamax = tf.keras.optimizers.Adamax(lr=0.002,beta_1=0.9,beta_2=0.999,epsilon=None,decay=0.0)
nadam = tf.keras.optimizers.Nadam(lr=0.002,beta_1=0.9,beta_2=0.999,epsilon=None,schedule_decay=0.004)
rms = tf.keras.optimizers.RMSprop(lr=0.001,rho=0.9,epsilon=None,decay=0.0)

# Losses https://keras.io/losses/
loss = ['sparse_categorical_crossentropy','binary_crossentropy','mean_squared_error','mean_absolute_error',
        'categorical_crossentropy','categorical_hinge']

# Metrics  https://www.tensorflow.org/api_docs/python/tf/metrics
metrics = ['accuracy','precision','recall']

In [ ]:
# run an experiment
repeats = 5
# load data
trainX, trainy, testX, testy = load_dataset(data_dir)

# repeat experiment
scores = list()
for r in range(repeats):
  score, model = evaluate_model(trainX, trainy, testX, testy)
  score = score * 100.0
  print('Accuracy at round #%d: %.3f' % (r+1, score))
  scores.append(score)

# summarize results
m, s = mean(scores), std(scores)
print('Accuracy: %.3f%% (+/-%.3f)' % (m, s))
model.summary()